In [ ]:
import os
import copy

import numpy as np
import torch
import torch.nn as nn
import math

from torch import optim
from torch.utils.data import Dataset, DataLoader
from torch.nn import functional as F
from unet import UNet
from dice_loss import dice_coeff

####################################################
# for data preparation
####################################################
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.metrics import roc_auc_score, accuracy_score, balanced_accuracy_score
####################################################
# for plotting
####################################################
import matplotlib.pyplot as plt
from IPython.display import clear_output
############################
# Helper func
############################
from helper import * 
#################################
TRAIN_RATIO = 0.8
RS = 30448 # random state
N_CHANNELS, N_CLASSES = 1, 1 
bilinear = True
BATCH_SIZE, EPOCHS = 16, 300
IMAGE_SIZE = (256, 256)
CROP_SIZE = (224, 224)
#########################################
DIR = 'dataset/3datasets_segment_v2'
CLIENTS = ['BUS', 'BUSIS', 'UDIAT']
CLIENTS_2 = [cl +'_2' for cl in CLIENTS]
TOTAL_CLIENTS = len(CLIENTS)
#####################################
# add the classification segment ####
#####################################
DIR_CLASSIFICATION = DIR + '/BUS/classification'

In [ ]:
device = torch.device('cuda:0')
LR, WD, TH = 1e-3, 1e-5, 0.9

## Training path - Testing path

In [ ]:
consistent_path = np.load('dict_path.npy', allow_pickle=True).item()
breast_dataset = dict()
for client in CLIENTS:
    print("loading data from ", client)
    dir_of_interest = consistent_path[client]
    x_train = dir_of_interest['x_train']
    x_test = dir_of_interest['x_test']
    y_train = dir_of_interest['y_train']
    y_test = dir_of_interest['y_test']
    
    # add normal images for bus # 
    DIR_INTEREST = DIR + '/'+ client 
    DATA_TYPE = ['original', 'GT']
    if client == 'BUS':
        for _,_, files in os.walk(DIR_INTEREST +'/classification/GT'):
            selected = [f for f in files if f[:6] =='normal']
            # update accordingly #
            for data in DATA_TYPE:
                tmp = [DIR_INTEREST + '/classification/' + data + '/' + f for f in selected]
                if data == 'GT':
                    y_train += tmp
                else:
                    x_train += tmp
            print(x_train)
    
    # to measure the weight # 
    
    breast_dataset[client+'_train']=Cancer(x_train, y_train, train=True,\
                                          IMAGE_SIZE=IMAGE_SIZE\
                                           , CROP_SIZE=CROP_SIZE)
    
    breast_dataset[client+'_test'] =Cancer(x_test, y_test, train=False,\
                                          IMAGE_SIZE=IMAGE_SIZE\
                                           , CROP_SIZE=CROP_SIZE)
    

# storage file

In [ ]:
training_clients, testing_clients = dict(), dict()
training_clients_pl = dict()

acc_train, acc_test, loss_train, loss_test = dict(), dict(), \
                                            dict(), dict()
    
nets, optimizers = dict(), dict()

In [ ]:
nets['global'] = UNet(n_channels=N_CHANNELS, n_classes=N_CLASSES, \
                      bilinear=True).to(device)

nets['global_2'] = UNet(n_channels=N_CHANNELS, n_classes=N_CLASSES, \
                      bilinear=True).to(device)

for client in CLIENTS:
    training_clients[client] = DataLoader(breast_dataset[client+'_train'], batch_size=16,\
                 shuffle=True, num_workers=8)
    training_clients_pl[client] = DataLoader(breast_dataset[client+'_train'], batch_size=1, \
                shuffle=True, num_workers=8)
    ###################################################################################
    testing_clients[client] = DataLoader(breast_dataset[client+'_test'], batch_size=1,\
                         shuffle=False, num_workers=1)
    
    acc_train[client], acc_test[client] = [], []
    loss_train[client], loss_test[client] = [], []
        
    nets[client] = UNet(n_channels=N_CHANNELS, n_classes=N_CLASSES, \
                      bilinear=True).to(device)
    nets[client+'_2'] = UNet(n_channels=N_CHANNELS, n_classes=N_CLASSES, \
                      bilinear=True).to(device)
    optimizers[client]= optim.Adam(nets[client].parameters(), \
                                   lr=LR,weight_decay=WD)
    optimizers[client+'_2']= optim.Adam(nets[client+'_2'].parameters(), \
                                   lr=LR,weight_decay=WD)

# local learning

In [ ]:
CLIENTS_SUPERVISION = ['labeled', 'labeled', 'labeled']

In [ ]:
best_avg_acc, best_epoch_avg = 0, 0
index = []

for client in CLIENTS:
    acc_train[client], acc_test[client] = [], []
    loss_train[client], loss_test[client] = [], []

for epoch in range(EPOCHS):
    index.append(epoch)
    #### conduct training #####
    for client, supervision_t in zip(CLIENTS, CLIENTS_SUPERVISION):
        bbox_supervision = False
        train_model(training_clients[client], nets[client], \
                                  optimizers[client], device, \
                                  acc = acc_train[client], \
                                  loss = loss_train[client], \
                                  supervision_type = supervision_t)
       
    ################### test ##############################
    avg_acc = 0.0
    for client in CLIENTS:
        test(epoch, testing_clients[client], nets[client], device, acc_test[client],\
             loss_test[client])
        avg_acc += acc_test[client][-1]
        
    avg_acc = avg_acc / TOTAL_CLIENTS
    ############################################################
    ########################################################
    if avg_acc > best_avg_acc:
        best_avg_acc = avg_acc
        best_epoch = epoch
    
    ################################
    # plot #########################
    ################################
    clear_output(wait=True)
    print(avg_acc, best_avg_acc)
    plt.figure(0)
    plt.plot(index, acc_train['UDIAT'], colors[0], label='UDIAT train')
    plt.plot(index, acc_train['BUS'], colors[1], label='BUS  train')
    plt.plot(index, acc_train['BUSIS'], colors[3], label='BUSIS  train')

    plt.legend()
    plt.show()

    plt.figure(1)
    plt.plot(index, loss_train['UDIAT'], colors[0], label='UDIAT loss train')
    plt.plot(index, loss_train['BUS'], colors[1], label='BUS  loss train')
    plt.plot(index, loss_train['BUSIS'], colors[3], label='BUSIS  loss train')
    plt.legend()
    plt.show()
    
    plot_graphs(2, CLIENTS, index, acc_test, ' acc_test')

print(best_avg_acc, best_epoch)
for client in CLIENTS:
    print(client)
    tmp = best_epoch
    best_epoch = best_epoch 
    print("shared epoch specific")
    print(acc_test[client][best_epoch])
    print("max client-specific")
    print(np.max(acc_test[client]))
    best_epoch = tmp